In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aviakompaniya/sample_submission.csv
/kaggle/input/aviakompaniya/train_dataset.csv
/kaggle/input/aviakompaniya/test_dataset.csv


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [3]:
train_df = pd.read_csv("/kaggle/input/aviakompaniya/train_dataset.csv")
test_df = pd.read_csv("/kaggle/input/aviakompaniya/test_dataset.csv")

In [4]:
train_df.head()

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,1,Male,disloyal Customer,33,Business travel,Eco,571,2,3,2,...,4,3,1,3,4,3,4,10,3.0,0
1,2,Female,Loyal Customer,49,Business travel,Business,1431,4,1,4,...,5,5,5,5,3,5,3,0,0.0,1
2,3,Female,Loyal Customer,43,Business travel,Eco,867,1,4,4,...,1,1,1,1,1,1,2,0,18.0,0
3,4,Female,Loyal Customer,27,Business travel,Business,1550,3,3,3,...,2,4,4,5,5,4,2,0,0.0,1
4,5,Male,Loyal Customer,11,Personal Travel,Eco,526,3,4,3,...,4,5,2,5,3,5,4,0,10.0,0


In [5]:
train_df.describe()

,id,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,9972.000000,10000.000000
mean,5000.50000,39.375100,1229.556200,2.780200,3.065100,2.797800,2.981000,3.214500,3.349100,3.49140,3.402900,3.454100,3.415700,3.676200,3.360600,3.680100,3.311100,14.429900,14.709888,0.500000
std,2886.89568,14.897959,1022.281958,1.362745,1.527316,1.421168,1.283278,1.333217,1.352779,1.31122,1.331822,1.274777,1.308765,1.189746,1.258462,1.175118,1.311217,39.659674,40.046355,0.500025
min,1.00000,7.000000,31.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.00000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2500.75000,27.000000,421.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.00000,2.000000,3.000000,2.000000,3.000000,3.000000,3.000000,2.000000,0.000000,0.000000,0.000000
50%,5000.50000,40.000000,863.500000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,4.00000,4.000000,4.000000,4.000000,4.000000,3.000000,4.000000,3.000000,0.000000,0.000000,0.500000
75%,7500.25000,51.000000,1806.750000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,5.00000,5.000000,4.000000,5.000000,5.000000,4.000000,5.000000,4.000000,12.000000,12.000000,1.000000
max,10000.00000,80.000000,4983.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,921.000000,924.000000,1.000000


In [6]:
train_df.isnull().sum()

id                                    0
Gender                                0
Customer Type                         0
Age                                   0
Type of Travel                        0
Class                                 0
Flight Distance                       0
Inflight wifi service                 0
Departure/Arrival time convenient     0
Ease of Online booking                0
Gate location                         0
Food and drink                        0
Online boarding                       0
Seat comfort                          0
Inflight entertainment                0
On-board service                      0
Leg room service                      0
Baggage handling                      0
Checkin service                       0
Inflight service                      0
Cleanliness                           0
Departure Delay in Minutes            0
Arrival Delay in Minutes             28
satisfaction                          0
dtype: int64

### Preprocessing Data

#### Convering categorical values to numerical values + filling missing values

In [7]:
def preprocess(data):
    # Filling missing values
    median_arrival_delay = data['Arrival Delay in Minutes'].median()
    data['Arrival Delay in Minutes'] = data['Arrival Delay in Minutes'].fillna(median_arrival_delay)

    sex_mapping = {'Male': 0, 'Female': 1}
    customer_mapping = {'disloyal Customer': 0, 'Loyal Customer': 1}
    travel_mapping = {'Personal Travel': 1, 'Business travel': 2}
    class_mapping = {'Eco': 1, 'Eco Plus': 2, 'Business':3}
    data['Gender'] = data['Gender'].map(sex_mapping)
    data['Customer Type'] = data['Customer Type'].map(customer_mapping)
    data['Type of Travel'] = data['Type of Travel'].map(travel_mapping)
    data['Class'] = data['Class'].map(class_mapping)

    
preprocess(train_df)

#### Plotting to see distribution

In [8]:
# fig, axes = plt.subplots(nrows=4, ncols=5, figsize=(20, 15))
# axes = axes.ravel()
# for idx, ax in enumerate(axes):
#     ax.hist(train_df.iloc[:, idx], bins=40, color='r')
#     ax.set_title(train_df.columns[idx])
# plt.tight_layout()
# plt.show()


#### Scaling Age and Flight Distance

In [9]:
def scale(data):
  columns_to_scale = ['Age', 'Flight Distance']
  scaler = StandardScaler()
  data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

scale(train_df)

In [12]:
# print(abs(train_df.corrwith(train_df.satisfaction)).sort_values(ascending=False))
# sns.heatmap(train_df.corr())
# plt.show()

### Splitting to Train and Test set

In [13]:
train_set, test_set = train_test_split(train_df, test_size=0.2, random_state=42)
X_train=train_set.drop('satisfaction', axis=1)
y_train=train_set['satisfaction']
X_test=test_set.drop('satisfaction', axis=1)
y_test=test_set['satisfaction']

### Decision Tree

In [14]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
y_pred = decision_tree.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.924


#### Finding the optimal hyperparameters with GridSearchCV

In [17]:
# param_grid = {
#     'max_depth':[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
#     'min_samples_split': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
# }

# dt_classifier = DecisionTreeClassifier(random_state=42)

# grid_search = GridSearchCV(dt_classifier, param_grid, cv=5)
# grid_search.fit(X_train, y_train)

# # Get the best hyperparameters
# best_params = grid_search.best_params_
# print("Best hyperparameters:", best_params)

# # Evaluate the model with the best hyperparameters
# best_dt_classifier = grid_search.best_estimator_
# y_pred = best_dt_classifier.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)

In [18]:
# Best hyperparameters: {'max_depth': 12, 'min_samples_split': 14}
decision_tree_model = DecisionTreeClassifier(min_samples_split =14,
                                             max_depth =12,).fit(X_train,y_train)

In [19]:
print(f"Metrics train:\n\tAccuracy score: {accuracy_score(decision_tree_model.predict(X_train),y_train):.4f}")
print(f"Metrics validation:\n\tAccuracy score: {accuracy_score(decision_tree_model.predict(X_test),y_test):.4f}")

Metrics train:
	Accuracy score: 0.9656
Metrics validation:
	Accuracy score: 0.9285


### RandomForestTree 

In [20]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9485


#### Finding the optimal hyperparameters with GridSearchCV

In [21]:
# param_grid = {
#     'n_estimators': [10, 50, 100, 150],
#     'max_depth': [5, 10, 20, 30],
#     'min_samples_split': [2, 5, 10, 15],
#     'min_samples_leaf': [1, 2, 4, 6],
# }

# rf_classifier = RandomForestClassifier(random_state=42)

# grid_search = GridSearchCV(rf_classifier, param_grid, cv=5)
# grid_search.fit(X_train, y_train)

# # Get the best hyperparameters
# best_params = grid_search.best_params_
# print("Best hyperparameters:", best_params)

# # Evaluate the model with the best hyperparameters
# best_rf_classifier = grid_search.best_estimator_
# y_pred = best_rf_classifier.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)

In [22]:
# Best hyperparameters: {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 50}
random_forest_model = RandomForestClassifier(n_estimators = 50,
                                             max_depth = 30, 
                                             min_samples_leaf = 1,
                                             min_samples_split= 10).fit(X_train,y_train)

In [23]:
print(f"Metrics train:\n\tAccuracy score: {accuracy_score(random_forest_model.predict(X_train),y_train):.4f}")
print(f"Metrics validation:\n\tAccuracy score: {accuracy_score(random_forest_model.predict(X_test),y_test):.4f}")

Metrics train:
	Accuracy score: 0.9792
Metrics validation:
	Accuracy score: 0.9415


## Test data

In [24]:
preprocess(test_df)
scale(test_df)

In [25]:
y_pred = random_forest_model.predict(test_df)

In [27]:
submission = pd.read_csv("/kaggle/input/aviakompaniya/sample_submission.csv")

In [29]:
submission.satisfaction = y_pred

In [30]:
submission

,id,satisfaction
0,1,0
1,2,1
2,3,1
3,4,1
4,5,0
...,...,...
3995,3996,1
3996,3997,0
3997,3998,0
3998,3999,1


In [31]:
submission.to_csv('submission.csv')